In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# import the required libraries for web scraping
from bs4 import BeautifulSoup
import requests
import re
import time
import random

In [ ]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 200)

In [ ]:
dota_players = pd.read_csv('data/dota_players.csv')
dota_teams = pd.read_csv('data/dota_teams.csv')
dota_heroes = pd.read_csv('data/dota_heroes.csv')

In [ ]:
single_performances = pd.read_csv('data/datdota_singleperformances.csv')

In [ ]:
single_performances

In [ ]:
# get a list of how gpm scales with kills and last hits
gpm_kills = single_performances.groupby('Kills')['GPM'].mean()
gpm_lh = single_performances.groupby('Last Hits')['GPM'].mean()
xpm_kills = single_performances.groupby('Kills')['XPM'].mean()
xpm_lh = single_performances.groupby('Last Hits')['XPM'].mean()
xpm_level = single_performances.groupby('LVL')['XPM'].mean()

# plot the data
fig, ax = plt.subplots(figsize=(10, 6))
fig2, ax2 = plt.subplots(figsize=(10, 6))
ax.plot(gpm_kills.index, gpm_kills.values, label='GPM per kill')
ax2.plot(gpm_lh.index, gpm_lh.values, label='GPM per last hit')
ax.set_xlabel('Kills')
ax.set_ylabel('GPM')
ax2.set_xlabel('Last Hits')
ax2.set_ylabel('GPM')
ax2.legend()
ax.legend()
plt.show()

In [ ]:
# drop the major outliers in single_performances
single_performances = single_performances[single_performances['Kills'] < 30]


In [ ]:
# get the value ranges for all columns in single_performances sorted by hero, ignoring Match, Player, Result, and End Game Items
hero_stats = single_performances.drop(['Match', 'Player', 'Result', 'End Game Items', 'KDA'], axis=1).groupby('Hero').agg(['min', 'max', 'mean'])


In [ ]:
# get the value ranges for all columns in single_performances and put them in a dictionary
value_ranges = {}
value_averages = {}
for col in single_performances.columns:
    if col != 'Match' and col != 'Hero' and col != 'Player' and col != 'Result' and col != 'End Game Items':
        value_ranges[col] = [min(single_performances[col]), max(single_performances[col])]
        value_averages[col] = np.mean(single_performances[col])
value_ranges

In [ ]:
player_options = dota_players['ID'].unique()

In [41]:
# a class to store single performance data
class SinglePerformance:
    def __init__(
        self,
        match_id,
        player_id,
        hero_id,
        kills,
        deaths,
        assists,
        last_hits,
        denies,
        gpm,
        xpm,
        hero_damage,
        tower_damage,
        hero_healing,
        level,
    ):
        self.match_id = match_id
        self.player_id = player_id
        self.hero_id = hero_id
        self.kills = kills
        self.deaths = deaths
        self.assists = assists
        self.last_hits = last_hits
        self.denies = denies
        self.gpm = gpm
        self.xpm = xpm
        self.hero_damage = hero_damage
        self.tower_damage = tower_damage
        self.hero_healing = hero_healing
        self.level = level

    def get_info(self):
        return {
            "Match": self.match_id,
            "Player": self.player_id,
            "Hero": self.hero_id,
            "Kills": self.kills,
            "Deaths": self.deaths,
            "Assists": self.assists,
            "Last_Hits": self.last_hits,
            "Denies": self.denies,
            "GPM": self.gpm,
            "XPM": self.xpm,
            "Hero_Damage": self.hero_damage,
            "Tower_Damage": self.tower_damage,
            "Hero_Healing": self.hero_healing,
            "Level": self.level,
        }

In [77]:
hero_stats

Hero Kills                Deaths                Assists  \
                           min max       mean    min max       mean     min   
0                Abaddon     0   2   1.000000      2   5   3.500000       4   
1     Ancient Apparition     3   3   3.000000      6   6   6.000000      16   
2              Anti-Mage    10  10  10.000000      0   0   0.000000       9   
3             Arc Warden     3  10   5.750000      0   5   2.250000       7   
4                    Axe     5  13   9.333333      4  11   8.333333       2   
5                   Bane     0   3   1.500000      5  10   7.500000       6   
6               Batrider     1  16   7.980769      0  11   4.288462       1   
7            Beastmaster     0   9   3.348837      0  10   4.395349       1   
8            Bloodseeker     0  16   5.266667      0   9   4.333333       0   
9          Bounty Hunter     0   4   1.666667      3   8   5.333333       4   
10            Brewmaster     0   4   2.600000      1   6   3.800000       5   
11           Bristleback     5  13   8.750000      1   1   1.000000       7   
12           Broodmother     0  14   4.425000      0   9   2.950000       0   
13     Centaur Warrunner     0  12   3.833333      0   8   4.291667       2   
14          Chaos Knight     1  18   6.900000      0  10   4.450000       0   
15                  Chen     0   3   1.000000      0   7   3.250000       2   
16                Clinkz    18  18  18.000000      1   1   1.000000      13   
17             Clockwerk     0   8   2.617647      1  17   7.779412       0   
18        Crystal Maiden     0   8   3.500000      2  16   8.000000       2   
19             Dark Seer     0   8   4.230769      1   9   4.769231       1   
20           Dark Willow     0  16   4.444444      1  12   6.777778       5   
21           Dawnbreaker     0  10   3.800000      0  14   5.350000       1   
22         Death Prophet     0  19   6.162791      0  12   4.081395       0   
23             Disruptor     0   7   2.575758      1  14   6.575758       3   
24                  Doom     0  16   5.093750      0  13   5.250000       1   
25         Dragon Knight     1  17   6.235294      2   9   4.941176       5   
26           Drow Ranger     0  22   8.000000      0   8   3.678571       1   
27          Earth Spirit     0  13   3.000000      4  10   8.000000       7   
28           Earthshaker     1  10   5.500000      2   8   4.250000      11   
29           Elder Titan     0   1   0.800000      2  10   5.600000      10   
30          Ember Spirit     0  27   8.323529      0  13   3.852941       1   
31           Enchantress     0  11   3.705882      0  13   6.529412       0   
32                Enigma     0   7   3.192308      1  12   4.500000       1   
33         Faceless Void     1  17   8.266667      0   9   4.533333       3   
34            Grimstroke     0   2   1.250000      5   7   6.250000       4   
35            Gyrocopter     2   4   3.000000      6   7   6.500000       1   
36              Hoodwink     0  10   3.796296      1  11   5.629630       2   
37                Huskar     6  16  10.333333      2   6   3.666667       9   
38               Invoker     0  19   8.714286      0   6   2.761905       0   
39                    Io     0   3   1.000000      1   8   4.333333       6   
40                Jakiro     0   3   1.714286      2  11   6.428571       6   
41            Juggernaut     6   6   6.000000      3   3   3.000000       8   
42   Keeper of the Light     0  15   5.656250      0  10   3.968750       1   
43                Kunkka     0  12   4.583333      0   8   3.250000       5   
44      Legion Commander     0   9   2.500000      4  11   6.500000       1   
45               Leshrac     0  20   8.595745      0  12   3.659574       0   
46                  Lich     1   8   3.111111      1  15   6.555556       5   
47           Lifestealer     0  10   4.600000      0   6   2.500000       4   
48                  Lina     1  24   9.206897      0  11   3.586207   

In [72]:
# a function to create random match data. it accepts an array of player ids. it returns a list of 10 SinglePerformance objects. The hero ids are randomly selected from the list of heroes, and if a hero has already been chosen, choose a new one. The stats are generated based on the hero values in hero_stats.
def create_random_match(players):
    match_id = random.randint(100000, 999999)
    performances = []
    for player in players:
        hero_id = random.choice(dota_heroes['HERO_ID'])
        while hero_id in [x.hero_id for x in performances]:
            hero_id = random.choice(dota_heroes['HERO_ID'])
        kills = random.randint(hero_stats.loc[hero_id]['Kills']['min'], hero_stats.loc[hero_id]['Kills']['max'])
        deaths = random.randint(hero_stats.loc[hero_id]['Deaths']['min'], hero_stats.loc[hero_id]['Deaths']['max'])
        assists = random.randint(hero_stats.loc[hero_id]['Assists']['min'], hero_stats.loc[hero_id]['Assists']['max'])
        last_hits = random.randint(hero_stats.loc[hero_id]['Last Hits']['min'], hero_stats.loc[hero_id]['Last Hits']['max'])
        denies = random.randint(hero_stats.loc[hero_id]['Denies']['min'], hero_stats.loc[hero_id]['Denies']['max'])
        gpm = random.randint(hero_stats.loc[hero_id]['GPM']['min'], hero_stats.loc[hero_id]['GPM']['max'])
        xpm = random.randint(hero_stats.loc[hero_id]['XPM']['min'], hero_stats.loc[hero_id]['XPM']['max'])
        hero_damage = random.randint(hero_stats.loc[hero_id]['HD']['min'], hero_stats.loc[hero_id]['HD']['max'])
        tower_damage = random.randint(hero_stats.loc[hero_id]['TD']['min'], hero_stats.loc[hero_id]['TD']['max'])
        hero_healing = random.randint(hero_stats.loc[hero_id]['HH']['min'], hero_stats.loc[hero_id]['HH']['max'])
        level = random.randint(hero_stats.loc[hero_id]['LVL']['min'], hero_stats.loc[hero_id]['LVL']['max'])
        performances.append(SinglePerformance(match_id, player, hero_id, kills, deaths, assists, last_hits, denies, gpm, xpm, hero_damage, tower_damage, hero_healing, level))

    for performance in performances:
        performances[performances.index(performance)] = performance.get_info()
    return pd.DataFrame(performances)

In [73]:
randommatch = create_random_match(["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"])

In [74]:
randommatch

,Match,Player,Hero,Kills,Deaths,Assists,Last_Hits,Denies,GPM,XPM,Hero_Damage,Tower_Damage,Hero_Healing,Level
0,2951473646,A,112,4,5,5,181,17,378,549,13169,274,59,17
1,2951473646,B,31,2,7,22,132,4,479,541,13137,8264,5182,19
2,2951473646,C,52,8,1,10,349,13,482,628,19142,3401,5588,18
3,2951473646,D,65,1,6,16,21,2,228,284,4503,60,0,16
4,2951473646,E,99,11,4,12,573,3,426,545,41086,2345,8939,29
5,2951473646,F,49,3,6,10,34,6,284,594,6336,532,120,13
6,2951473646,G,21,8,7,7,269,14,442,760,16489,1328,2580,27
7,2951473646,H,56,5,4,3,809,23,552,714,99474,7407,0,17
8,2951473646,I,10,0,2,5,134,9,471,561,16426,532,249,26
9,2951473646,J,118,6,1,6,358,5,655,762,18654,10738,0,22


In [18]:
# create a dataframe of dataframes for each team_name in dota_players
team_dataframes = {}
for team_name in dota_players['team_name'].unique():
    team_dataframes[team_name] = dota_players[dota_players['team_name'] == team_name]

In [22]:
team_dataframes["Alliance"]

,Unnamed: 0,ID,name,country,Region,team_name,role
6,6,charlie,Charlie Arat,Sweden,Europe,Alliance,Carry
7,7,Loda,Jonathan Emanuel Berg,Sweden,Europe,Alliance,Coach
8,8,s4,Gustav Magnusson,Sweden,Europe,Alliance,Offlaner
9,9,ChYuan,Ng Kee Chyuan,Malaysia,Southeast Asia,Alliance,Solo Middle
10,11,ponlo,Remus Goh Zhi Xian,Singapore,Southeast Asia,Alliance,Support
11,12,Handsken,Simon Rasmus Haag,Sweden,Europe,Alliance,Support


In [ ]:
value_ranges

In [ ]:
regions = ['North America', 'Europe', 'China', 'Southeast Asia', 'South America', 'CIS']

In [ ]:
# function that takes a pandas dataframe and assign a region based on each player country
def assign_region(df):
    df['Region'] = df['country'].apply(lambda x: 'North America' if x in ['United States', 'Canada'] else 'Europe' if x in ['United Kingdom', 'Germany', 'France', 'Spain', 'Italy', 'Netherlands', 'Poland', 'Sweden', 'Denmark', 'Norway', 'Finland', 'Belgium', 'Switzerland', 'Austria', 'Portugal', 'Greece', 'Czech Republic', 'Hungary', 'Romania', 'Bulgaria', 'Serbia', 'Slovenia', 'Bosnia and Herzegovina', 'Croatia', 'Ireland', 'Luxembourg', 'Slovakia', 'Estonia', 'Latvia', 'Lithuania', 'Moldova', 'Montenegro', 'Albania', 'Cyprus', 'Macedonia', 'Malta', 'Ukraine', 'Armenia', 'Azerbaijan', 'Georgia', 'Kazakhstan', 'Russia', 'Turkey', 'Belarus', 'Iceland', 'Liechtenstein', 'Monaco', 'San Marino', 'Vatican City'] else 'China' if x in ['China', 'Hong Kong', 'Taiwan'] else 'Southeast Asia' if x in ['Singapore', 'Malaysia', 'Philippines', 'Indonesia', 'Thailand', 'Vietnam', 'Cambodia', 'Laos', 'Myanmar', 'Brunei'] else 'South America' if x in ['Brazil', 'Argentina', 'Chile', 'Colombia', 'Ecuador', 'Peru', 'Venezuela', 'Uruguay', 'Bolivia', 'Paraguay', 'Guyana', 'Suriname', 'French Guiana', 'Easter Island', 'Falkland Islands', 'South Georgia and the South Sandwich Islands', 'South Sandwich Islands', 'Antarctica'] else 'CIS' if x in ['Russia', 'Ukraine', 'Belarus', 'Kazakhstan', 'Azerbaijan', 'Armenia', 'Georgia', 'Kyrgyzstan', 'Moldova', 'Tajikistan', 'Turkmenistan', 'Uzbekistan', 'Kosovo', 'Abkhazia', 'South Ossetia'] else 'Other')
    return df

In [ ]:
dota_players = assign_region(dota_active_players)

In [ ]:
# function that detects first occurence of char
def find_first(string, char):
    for i, c in enumerate(string):
        if c == char:
            return i
    return -1

# function that detects last occurence of char
def find_last(string, char):
    for i, c in enumerate(string[::-1]):
        if c == char:
            return len(string) - i - 1
    return -1

In [ ]:
# split the team column into team name and role
dota_players['team_name'] = dota_players['team'].apply(lambda x: x[:find_first(x, '(')].strip())
dota_players['role'] = dota_players['team'].apply(lambda x: x[find_first(x, '(')+1:find_last(x, ')')])
dota_players.drop('team', axis=1, inplace=True)

In [ ]:
roles = ['Support', 'Offlaner', 'Solo Middle', 'Carry', 'Coach']

In [ ]:
# filter the roles
dota_players = dota_players[dota_players['role'].isin(roles)]

### Below is for scraping dota data from liquipedia

In [ ]:
# scrape the data from the website
url = "https://liquipedia.net/dota2/Players_(all)"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
rows = soup.findAll("tr")
# # find the table with the data
# table = soup.find('table', class_='wikitable')

In [ ]:
rows[1]

In [ ]:
indexes = rows[1]
index_values = []
for cell in indexes.find_all("th"):
    index_values.append(cell.get_text().rstrip())
players = []

In [ ]:
index_values

In [ ]:
for row in rows:
    if len(row) > 3:
        player = {}
        cells = row.find_all("td")
        for i in range(0, len(cells)):
            key = index_values[i]
            if key == " Links":
                key = "country"
                value = cells[0].find("a").get("title")
            else:
                value = cells[i].get_text().rstrip()
            player[key] = value
        if len(player) > 0:
            players.append(player)

In [ ]:
player_df = pd.DataFrame(players)

In [ ]:
player_df.rename(
    columns={" Real Name": "name", " Team": "team", " ID": "ID"}, inplace=True
)

In [ ]:
filter_roles = {
    "Carry",
    "Mid",
    "Offlane",
    "Support",
    "Hard Support",
    "Soft Support",
    "Hard",
    "Soft",
    "Hard Carry",
    "Soft Carry",
    "Hard Carry",
    "Solo Middle",
    "Analyst",
    "Caster",
    "Host",
    "Offlaner"
}
# drop the rows that are in the filter_roles
player_df = player_df[~player_df["team"].isin(filter_roles)]

In [ ]:
player_df.to_csv("liquipedia_dota_players.csv", index=False)

In [ ]:
# strip all leading and trailing spaces
player_df = player_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
url = "https://liquipedia.net/dota2/Portal:Teams"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
teams = []
divs = soup.find_all('div',class_="panel-box")

In [ ]:


def get_teams():
	url = "https://liquipedia.net/dota2/Portal:Teams"
	page = requests.get(url)
	soup = BeautifulSoup(page.content, "html.parser")
	teams = []
	divs = soup.find_all('div',class_="panel-box")
	templates = soup.find_all('span',class_="team-template-team-standard")
	for team in templates:
		teams.append(team.a['title'])
			
	return teams

In [ ]:
teamdf = pd.DataFrame(get_teams())

In [ ]:
teamdf.to_csv("activeteams.csv")

In [ ]:
print(
    "Min: {}, Max: {}".format(
        dota_performances["GPM"].min(), dota_performances["GPM"].max()
    )
)